In [18]:
from time import time
from pearce.mocks.readGadgetSnapshot import readGadgetSnapshot
from fast3tree import fast3tree
import numpy as np
from halotools.sim_manager import RockstarHlistReader
from glob import glob
from os import path
from pearce.mocks.kittens import cat_dict

In [19]:
cat1 = cat_dict['chinchilla'](400.0, scale_factors=[0.658])
idx = np.argwhere(np.array(cat1.scale_factors)==0.658)

In [20]:
all_snapdirs = glob(path.join(cat1.gadget_loc, 'snapdir*'))

In [21]:
cat1.sf_idxs

array([83])

In [22]:
cat1.filenames

['/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//rockstar/hlists/hlist_0.65800.list']

In [23]:
snapdirs = [all_snapdirs[idx] for idx in cat1.sf_idxs]

In [24]:
snapdirs

['/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_083']

In [25]:
idx

83

In [26]:
cat = cat_dict['chinchilla'](400.0, scale_factors=[0.658])

In [27]:
snapdir = '/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034'

In [28]:
reader = RockstarHlistReader(cat.filenames[0], cat.columns_to_keep, cat.cache_filenames[0], cat.simname, cat.halo_finder, 0.5,\
                            cat.version_name, cat.Lbox, cat.pmass, overwrite=False)
reader.read_halocat(cat.columns_to_convert)



The information about your ascii file and the metadata about the catalog 
have been processed and no exceptions were raised. 
Use the ``read_halocat`` method to read the ascii data, 
setting the write_to_disk and update_cache_log arguments as you like. 
See the docstring of the ``read_halocat`` method
for details about these options. 


...Processing ASCII data of file: 
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048/rockstar/hlists/hlist_0.65800.list
 
Total number of rows in detected data = 19184048
Number of rows in detected header = 49 

... working on chunk 0 of 21
... working on chunk 1 of 21
... working on chunk 2 of 21
... working on chunk 3 of 21
... working on chunk 4 of 21
... working on chunk 5 of 21
... working on chunk 6 of 21
... working on chunk 7 of 21
... working on chunk 8 of 21
... working on chunk 9 of 21
... working on chunk 10 of 21
... working on chunk 11 of 21
... working on chunk 12 of 21
... working on chunk 13 of 21
... working on chun

In [29]:
t0 = time()
p = 1e-4
all_particles = np.array([], dtype='float32')
# TODO should fail gracefully if memory is exceeded or if p is too small.
t1 = time()
for file in glob(path.join(snapdir, 'snapshot*')):
    print file
    # TODO should find out which is "fast" axis and use that.
    # Numpy uses fortran ordering.
    particles = readGadgetSnapshot(file, read_pos=True)[1]  # Think this returns some type of tuple; should check
    particles = particles[np.random.rand(particles.shape[0]) < p]  # downsample
    if particles.shape[0] == 0:
        continue

    all_particles = np.resize(all_particles, (all_particles.shape[0] + particles.shape[0], 3))
    all_particles[-particles.shape[0]:, :] = particles

print 'All done', time()-t0, 's'

print all_particles.shape

/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034/snapshot_034.449
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034/snapshot_034.352
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034/snapshot_034.383
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034/snapshot_034.570
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034/snapshot_034.764
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034/snapshot_034.454
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034/snapshot_034.11
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034/snapshot_034.873
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test/c400-2048//output/snapdir_034/snapshot_034.12
/nfs/slac/g/ki/ki21/cosmo/yymao/sham_test/resolution-test

In [30]:
print all_particles.shape

(857439, 3)


In [31]:
print all_particles.mean(axis=0), all_particles.max(axis=0), all_particles.min(axis=0)

[ 200.22717285  200.29829407  200.73243713] [ 399.99963379  399.99954224  399.99951172] [  3.05175781e-04   0.00000000e+00   4.57658007e-06]


In [32]:
radius = 20
from itertools import izip

In [33]:
print len(reader.halo_table['halo_x'])

19184048


In [34]:
#all_pos *= h
#TODO not entirely sure if i'm applying little h correctly
#densities = np.ones(reader.halo_table['x'].shape)/(4*np.pi/(3*self.h**2)*radius**3)
densities = np.ones(reader.halo_table['halo_x'].shape)/(p*4*np.pi/3*reader.halo_table['halo_rvir']**3)
with fast3tree(all_particles) as tree:
    for idx, halo_pos in enumerate(izip(reader.halo_table['halo_x'],reader.halo_table['halo_y'],reader.halo_table['halo_z'], reader.halo_table['halo_rvir'])):
        #print idx, time()-t0, 's'
        if idx%1e6 == 0:
            print idx
        particle_idxs = tree.query_radius(halo_pos[:-1], halo_pos[-1], periodic=True)
        densities[idx]*=reader.particle_mass*len(particle_idxs)

reader.halo_table['halo_local_density'] = densities

0
1000000


KeyboardInterrupt: 

In [ ]:
densities.shape

In [ ]:
print densities.mean(), densities.max(), densities.min()

In [ ]:
np.log10(densities[densities>0]).mean()

In [ ]:
densities = np.loadtxt('halo_densities.npy')

In [ ]:
real_hd = reader.halo_table['halo_mvir']/(4*np.pi/3*reader.halo_table['halo_rvir']**3)

In [ ]:
mass_cut = reader.halo_table['halo_mvir']> 1e13

In [ ]:
real_hd = np.loadtxt('read_hd.npy')

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [ ]:
ratio = densities/real_hd

In [ ]:
print np.any(np.isnan(ratio))

In [ ]:
print ratio[mass_cut].mean(), ratio[mass_cut].std()

In [ ]:
plt.scatter(densities[mass_cut], real_hd[mass_cut])

In [ ]:
rand_idxs= np.random.choice(len(real_hd), size = 1e4, replace =False)

In [ ]:
sns.jointplot(real_hd[mass_cut], densities[mass_cut], kind = 'hex', space = 0)

In [ ]:
print real_hd.mean(), real_hd.std()

In [ ]:
plt.scatter(np.log10(real_hd[mass_cut]), np.log10(densities[mass_cut]),alpha=0.1)

In [ ]:
plt.hist(np.log10(real_hd) )

In [ ]:
plt.hist(np.log10(densities[densities> 0]) )